<span style="font-size: 150%;color:white;background:red"> MATERIAŁ WZORCOWY

<span style="font-size: 160%;color:#0000ff"> OCENA WYDAJNOŚCI PRACY PRACOWNIKÓW BANKU ZA POMOCĄ METODY DEA

<span style="font-size: 150%;color:white;background:#1155cc"> Data envelopment analysis (DEA)
    
Envelopment analiza danych (DEA), czasami nazywany analiza granica została po raz pierwszy przedstawiona przez Charnes, Cooper i Rodos w roku 1978. Jest to technika pomiaru wydajności, które, jak się przekonamy, może być stosowany do oceny względnej skuteczności w procesie decyzyjnym jednostek ( DMU ) w organizacjach. W tym przypadku DMU jest odrębną jednostką w organizacji, która ma elastyczność w odniesieniu do niektórych decyzji, które podejmuje, ale niekoniecznie pełną swobodę w odniesieniu do tych decyzji.

Przykładami takich jednostek, do których zastosowano DEA, są: banki, posterunki policji, szpitale, urzędy skarbowe, więzienia, bazy obronne (wojsko, marynarka wojenna, siły powietrzne), szkoły i wydziały uniwersyteckie. Należy zauważyć, że jedną z zalet DEA jest to, że można go zastosować do organizacji non-profit.

Od czasu pierwszego zaproponowania tej techniki wykonano wiele prac teoretycznych i empirycznych. Opublikowano wiele badań dotyczących stosowania DEA w rzeczywistych sytuacjach. Oczywiście niepublikowanych opracowań jest znacznie więcej, np. Wykonanych wewnętrznie przez firmy lub konsultantów zewnętrznych.

Na początku zilustrujemy DEA na małym przykładzie. Więcej o DEA można znaleźć tutaj . Zwróć uwagę, że wiele z tego, co zobaczysz poniżej, to graficzne (obrazowe) podejście do DEA. Jest to bardzo przydatne, jeśli próbujesz wyjaśnić DEA osobom mniej wykwalifikowanym technicznie (takim jak wielu, których możesz spotkać w świecie zarządzania). Istnieje jednak matematyczne podejście do DEA, które można jednak przyjąć - zostanie to zilustrowane poniżej.

http://people.brunel.ac.uk/~mastjjb/jeb/or/dea.html

Zadanie polega na tym, że analizuje się wydajność personelu pracującego w banku, wymyślono różne wskaźniki m.in. wskażnik ilości operacji bankowych na pracownika itp. 

<span style="font-size: 130%;color:#0000ff"> Jednym z problemów związanych z porównaniem za pomocą współczynników jest to, że różne współczynniki mogą dać inny obraz i trudno jest połączyć cały zestaw współczynników w jedną ocenę liczbową.
    
Na przykład rozważmy Dorking i Reigate - Dorking jest (2,75 / 2,09) = 1,32 razy skuteczniejszy niż Reigate w transakcjach osobistych, ale tylko (1,25 / 1,09) = 1,15 razy skuteczniejszy w transakcjach biznesowych. Jak połączyć te liczby w jedą ocenę?
Problem zacznie się gdy takich wskaźników dodamy jeszcze 5 czy 8. Jak wtedy ocenić? Przez średnią? 

<span style="font-size: 130%;color:#0000ff"> Graficznie w artykule mu wyszedł taki ranking:

 1. Croydon 100%
 2. Dorking 43%
 3. Redhill 100%
 4. Reigate 36%

In [1]:
from beautifultable import BeautifulTable

table = BeautifulTable()
table.column_headers = ["Bank","Transakcje osobiste" ,"Transakcje biznesowe"
, "Personel"]
table.append_row(["Croydon",    125,   50,     18 ])
table.append_row(["Dorking",    44,    20,     16 ])
table.append_row(["Redhill",    80,    55,     17 ])
table.append_row(["Reigate",    23,    12,     11 ])
print(table)

+---------+---------------------+----------------------+----------+
|  Bank   | Transakcje osobiste | Transakcje biznesowe | Personel |
+---------+---------------------+----------------------+----------+
| Croydon |         125         |          50          |    18    |
+---------+---------------------+----------------------+----------+
| Dorking |         44          |          20          |    16    |
+---------+---------------------+----------------------+----------+
| Redhill |         80          |          55          |    17    |
+---------+---------------------+----------------------+----------+
| Reigate |         23          |          12          |    11    |
+---------+---------------------+----------------------+----------+


Obliczam dla każdego oddziału banku oddzielnie programowanie liniowe

<span style="font-size: 230%;color:#0000ff"> Croydon

In [2]:
from pulp import *
from fractions import Fraction

# zdefiniowanie problemu- to jest taka platforma do wrzucania na nia wszystkiego

prob = LpProblem("DEA_Croydon",LpMaximize)

In [3]:
# "Continuous"
# "Integer"

x1=LpVariable("x1",lowBound=0,upBound=None, cat="Integer")
x2=LpVariable("x2",lowBound=0,upBound=None, cat="Integer")
x3=LpVariable("x3",lowBound=0,upBound=None, cat="Integer")

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU

<span style="font-size: 130%;color:#0000ff">$ {125x_{1}} + 50x_{2} \to max$

przy warunku, że dla Croydon:

<span style="font-size: 130%;color:black">$18x_{3}=1$

In [4]:
# objective function
prob += 125*x1 + 50*x2, "Maksymalizacja zysków"

In [5]:
print(prob)

DEA_Croydon:
MAXIMIZE
125*x1 + 50*x2 + 0
VARIABLES
0 <= x1 Integer
0 <= x2 Integer



##### $
\left\{
\begin{align} 
({125x_{1}} + 50x_{2})-18x_{3} & \leqslant 0& (1)\\
({44x_{1}} + 20x_{2})-16x_{3} & \leqslant 0& (2)\\
({80x_{1}} + 55x_{2})-17x_{3} & \leqslant 0& (3)\\
({23x_{1}} + 12x_{2})-11x_{3} & \leqslant 0& (4)\\
\end{align}
\right. 
$
Warunki znakowe:
#### $
\left\{
\begin{align} 
{x_{1}} & \geqslant 0& (5)\\
{x_{2}} & \geqslant 0& (6)\\
{x_{3}} & \geqslant 0& (7)\\
\end{align}
\right. 
$

$18x_{3}=1$              (8)

In [6]:
# main constraints
prob += (125*x1+50*x2)-18*x3 <= 0
prob += (44*x1+20*x2)-16*x3 <= 0
prob += (80*x1+55*x2)-17*x3 <= 0
prob += (23*x1+12*x2)-11*x3 <= 0
prob += 18*x3 == 1

In [7]:
print(prob)

DEA_Croydon:
MAXIMIZE
125*x1 + 50*x2 + 0
SUBJECT TO
_C1: 125 x1 + 50 x2 - 18 x3 <= 0

_C2: 44 x1 + 20 x2 - 16 x3 <= 0

_C3: 80 x1 + 55 x2 - 17 x3 <= 0

_C4: 23 x1 + 12 x2 - 11 x3 <= 0

_C5: 18 x3 = 1

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer



### Rozwiązanie problemu

In [8]:
print("prob.solve",prob.solve())
# status of the solution
print(f"Status: {LpStatus[prob.status]}")

prob.solve -1
Status: Infeasible


In [9]:
for v in prob.variables():
    print(f"{v.name} = {str(Fraction(v.varValue).limit_denominator())}")


x1 = 1/125
x2 = 0
x3 = 1/18


In [10]:
1/125

0.008

In [11]:
1/18

0.05555555555555555

In [12]:
(125*(1/125)+50*(0))/(18*(1/18))

1.0

Ocena Croydon jest bardzo dobra, 100%

Graficznie w artykule mu wyszedł taki ranking:

 1. Croydon 100%
 2. Dorking 43%
 3. Redhill 100%
 4. Reigate 36%

<span style="font-size: 230%;color:#0000ff"> Dorking

In [13]:
from pulp import *
from fractions import Fraction

# zdefiniowanie problemu- to jest taka platforma do wrzucania na nia wszystkiego

prob = LpProblem("DEA_Dorking",LpMaximize)

In [14]:
# "Continuous"
# "Integer"

x1=LpVariable("x1",lowBound=0,upBound=None, cat="Integer")
x2=LpVariable("x2",lowBound=0,upBound=None, cat="Integer")
x3=LpVariable("x3",lowBound=0,upBound=None, cat="Integer")

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU

<span style="font-size: 130%;color:#0000ff">$ {44x_{1}} + 20x_{2} \to max$
    
gdzie dla Dorking:

$17x_{3}=1$

In [15]:
# objective function
prob += 44*x1 + 20*x2, "Maksymalizacja zysków"

In [16]:
print(prob)

DEA_Dorking:
MAXIMIZE
44*x1 + 20*x2 + 0
VARIABLES
0 <= x1 Integer
0 <= x2 Integer



#### $
\left\{
\begin{align} 
({125x_{1}} + 50x_{2})-18x_{3} & \leqslant 0& (1)\\
({44x_{1}} + 20x_{2})-16x_{3} & \leqslant 0& (1)\\
({80x_{1}} + 55x_{2})-17x_{3} & \leqslant 0& (1)\\
({23x_{1}} + 12x_{2})-11x_{3} & \leqslant 0& (1)\\
\end{align}
\right. 
$

#### $
\left\{
\begin{align} 
{x_{1}} & \geqslant 0& (1)\\
{x_{2}} & \geqslant 0& (2)\\
{x_{3}} & \geqslant 0& (3)\\
\end{align}
\right. 
$

$16x_{3}=1$

In [17]:
# main constraints
prob += (125*x1+50*x2)-18*x3 <= 0
prob += (44*x1+20*x2)-16*x3 <= 0
prob += (80*x1+55*x2)-17*x3 <= 0
prob += (23*x1+12*x2)-11*x3 <= 0
prob += 16*x3 == 1

In [18]:
print(prob)

DEA_Dorking:
MAXIMIZE
44*x1 + 20*x2 + 0
SUBJECT TO
_C1: 125 x1 + 50 x2 - 18 x3 <= 0

_C2: 44 x1 + 20 x2 - 16 x3 <= 0

_C3: 80 x1 + 55 x2 - 17 x3 <= 0

_C4: 23 x1 + 12 x2 - 11 x3 <= 0

_C5: 16 x3 = 1

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer



### Rozwiązanie problemu

In [19]:
print("prob.solve",prob.solve())
# status of the solution
print(f"Status: {LpStatus[prob.status]}")

prob.solve -1
Status: Infeasible


In [20]:
for v in prob.variables():
    print(f"{v.name} = {str(Fraction(v.varValue).limit_denominator())}")


x1 = 7/2300
x2 = 4671/313673
x3 = 1/16


In [21]:
x1 = 7/2300
x2 = 4671/313673
x3 = 1/16

$({44x_{1}} + 20x_{2})/16x_{3}$

In [22]:
(44*x1+20*x2)/(16*x3)

0.43173912350427524

Dorking jest słaby, 43%

<span style="font-size: 130%;color:#0000ff"> Graficznie w artykule mu wyszedł taki ranking:

 1. Croydon 100%
 2. Dorking 43%
 3. Redhill 100%
 4. Reigate 36%

<span style="font-size: 230%;color:#0000ff"> Redhill

In [23]:
from beautifultable import BeautifulTable

table = BeautifulTable()
table.column_headers = ["Bank","Transakcje osobiste" ,"Transakcje biznesowe"
, "Personel"]
table.append_row(["Croydon",    125,   50,     18 ])
table.append_row(["Dorking",    44,    20,     16 ])
table.append_row(["Redhill",    80,    55,     17 ])
table.append_row(["Reigate",    23,    12,     11 ])
print(table)

+---------+---------------------+----------------------+----------+
|  Bank   | Transakcje osobiste | Transakcje biznesowe | Personel |
+---------+---------------------+----------------------+----------+
| Croydon |         125         |          50          |    18    |
+---------+---------------------+----------------------+----------+
| Dorking |         44          |          20          |    16    |
+---------+---------------------+----------------------+----------+
| Redhill |         80          |          55          |    17    |
+---------+---------------------+----------------------+----------+
| Reigate |         23          |          12          |    11    |
+---------+---------------------+----------------------+----------+


In [24]:
from pulp import *
from fractions import Fraction

# zdefiniowanie problemu- to jest taka platforma do wrzucania na nia wszystkiego

prob = LpProblem("DEA_Redhill",LpMaximize)

In [25]:
# "Continuous"
# "Integer"

x1=LpVariable("x1",lowBound=0,upBound=None, cat="Integer")
x2=LpVariable("x2",lowBound=0,upBound=None, cat="Integer")
x3=LpVariable("x3",lowBound=0,upBound=None, cat="Integer")

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU

<span style="font-size: 130%;color:#0000ff">$ {80x_{1}} + 55x_{2} \to max$

In [26]:
# objective function
prob += 80*x1 + 55*x2, "Maksymalizacja zysków"

In [27]:
print(prob)

DEA_Redhill:
MAXIMIZE
80*x1 + 55*x2 + 0
VARIABLES
0 <= x1 Integer
0 <= x2 Integer



#### $
\left\{
\begin{align} 
({125x_{1}} + 50x_{2})-18x_{3} & \leqslant 0& (1)\\
({44x_{1}} + 20x_{2})-16x_{3} & \leqslant 0& (1)\\
({80x_{1}} + 55x_{2})-17x_{3} & \leqslant 0& (1)\\
({23x_{1}} + 12x_{2})-11x_{3} & \leqslant 0& (1)\\
\end{align}
\right. 
$

#### $
\left\{
\begin{align} 
{x_{1}} & \geqslant 0& (1)\\
{x_{2}} & \geqslant 0& (2)\\
{x_{3}} & \geqslant 0& (3)\\
\end{align}
\right. 
$

$17x_{3}=1$

In [28]:
# main constraints
prob += (125*x1+50*x2)-18*x3 <= 0
prob += (44*x1+20*x2)-16*x3 <= 0
prob += (80*x1+55*x2)-17*x3 <= 0
prob += (23*x1+12*x2)-11*x3 <= 0
prob += 17*x3 == 1

In [29]:
print(prob)

DEA_Redhill:
MAXIMIZE
80*x1 + 55*x2 + 0
SUBJECT TO
_C1: 125 x1 + 50 x2 - 18 x3 <= 0

_C2: 44 x1 + 20 x2 - 16 x3 <= 0

_C3: 80 x1 + 55 x2 - 17 x3 <= 0

_C4: 23 x1 + 12 x2 - 11 x3 <= 0

_C5: 17 x3 = 1

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer



### Rozwiązanie problemu

In [30]:
print("prob.solve",prob.solve())
# status of the solution
print(f"Status: {LpStatus[prob.status]}")

prob.solve -1
Status: Infeasible


In [31]:
for v in prob.variables():
    print(f"{v.name} = {str(Fraction(v.varValue).limit_denominator())}")


x1 = 28/9775
x2 = 5323/379798
x3 = 1/17


Kopiuje z góry

In [32]:
x1 = 28/9775
x2 = 5323/379798
x3 = 1/17

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU

<span style="font-size: 130%;color:#0000ff">$ {80x_{1}} + 55x_{2} \to max$

In [33]:
(80*x1+55*x2)/(17*x3)

0.9999999851852868

Redhill 100% jest bardzo dobry!

<span style="font-size: 130%;color:#0000ff"> Graficznie w artykule mu wyszedł taki ranking:

 1. Croydon 100%
 2. Dorking 43%
 3. Redhill 100%
 4. Reigate 36%

<span style="font-size: 230%;color:#0000ff"> Reigate

In [34]:
from beautifultable import BeautifulTable

table = BeautifulTable()
table.column_headers = ["Bank","Transakcje osobiste" ,"Transakcje biznesowe"
, "Personel"]
table.append_row(["Croydon",    125,   50,     18 ])
table.append_row(["Dorking",    44,    20,     16 ])
table.append_row(["Redhill",    80,    55,     17 ])
table.append_row(["Reigate",    23,    12,     11 ])
print(table)

+---------+---------------------+----------------------+----------+
|  Bank   | Transakcje osobiste | Transakcje biznesowe | Personel |
+---------+---------------------+----------------------+----------+
| Croydon |         125         |          50          |    18    |
+---------+---------------------+----------------------+----------+
| Dorking |         44          |          20          |    16    |
+---------+---------------------+----------------------+----------+
| Redhill |         80          |          55          |    17    |
+---------+---------------------+----------------------+----------+
| Reigate |         23          |          12          |    11    |
+---------+---------------------+----------------------+----------+


In [35]:
from pulp import *
from fractions import Fraction

# zdefiniowanie problemu- to jest taka platforma do wrzucania na nia wszystkiego

prob = LpProblem("DEA_Reigate",LpMaximize)

In [36]:
# "Continuous"
# "Integer"

x1=LpVariable("x1",lowBound=0,upBound=None, cat="Integer")
x2=LpVariable("x2",lowBound=0,upBound=None, cat="Integer")
x3=LpVariable("x3",lowBound=0,upBound=None, cat="Integer")

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU Reigate

<span style="font-size: 130%;color:#0000ff">$ {23x_{1}} + 12x_{2} \to max$

przy warunku że:

$11x_{3}=1$

In [37]:
# objective function
prob += 23*x1 + 12*x2, "Maksymalizacja zysków"

In [38]:
print(prob)

DEA_Reigate:
MAXIMIZE
23*x1 + 12*x2 + 0
VARIABLES
0 <= x1 Integer
0 <= x2 Integer



#### $
\left\{
\begin{align} 
({125x_{1}} + 50x_{2})-18x_{3} & \leqslant 0& (1)\\
({44x_{1}} + 20x_{2})-16x_{3} & \leqslant 0& (1)\\
({80x_{1}} + 55x_{2})-17x_{3} & \leqslant 0& (1)\\
({23x_{1}} + 12x_{2})-11x_{3} & \leqslant 0& (1)\\
\end{align}
\right. 
$

#### $
\left\{
\begin{align} 
{x_{1}} & \geqslant 0& (1)\\
{x_{2}} & \geqslant 0& (2)\\
{x_{3}} & \geqslant 0& (3)\\
\end{align}
\right. 
$
dla Reigate: 
$11x_{3}=1$

In [39]:
# main constraints
prob += (125*x1+50*x2)-18*x3 <= 0
prob += (44*x1+20*x2)-16*x3 <= 0
prob += (80*x1+55*x2)-17*x3 <= 0
prob += (23*x1+12*x2)-11*x3 <= 0
prob += 11*x3 == 1

In [40]:
print(prob)

DEA_Reigate:
MAXIMIZE
23*x1 + 12*x2 + 0
SUBJECT TO
_C1: 125 x1 + 50 x2 - 18 x3 <= 0

_C2: 44 x1 + 20 x2 - 16 x3 <= 0

_C3: 80 x1 + 55 x2 - 17 x3 <= 0

_C4: 23 x1 + 12 x2 - 11 x3 <= 0

_C5: 11 x3 = 1

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer



### Rozwiązanie problemu

In [41]:
print("prob.solve",prob.solve())
# status of the solution
print(f"Status: {LpStatus[prob.status]}")

prob.solve -1
Status: Infeasible


In [42]:
for v in prob.variables():
    print(f"{v.name} = {str(Fraction(v.varValue).limit_denominator())}")


x1 = 28/6325
x2 = 137/6325
x3 = 1/11


Kopiuje z góry

In [43]:
x1 = 28/9775
x2 = 5323/379798
x3 = 1/17

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU

<span style="font-size: 130%;color:#0000ff">$ {23x_{1}} + 12x_{2} \to max$

In [44]:
(23*x1+12*x2)/(11*x3)

0.3617391254394082

<span style="font-size: 130%;color:#0000ff"> Graficznie w artykule mu wyszedł taki ranking:

 1. Croydon 100%
 2. Dorking 43%
 3. Redhill 100%
 4. Reigate 36%

# ZMIANA WAG
Ktoś z kierownictwa centrali banku uznał, że transakcie biznesowe są 12 razy ważniejsze niż transakcje zwykłe. 
To znaczy że jedna zawarta transakcja biznesowa liczy się tak jak 12 transakcji zwykłych.

$x_{1}$ _ Transakcje zwykłe 

$x_{2}$ _ Transakcje biznesowe

wtedy:

<span style="font-size: 230%;color:black">

$ \frac {x_{2}}{x_{1}} >= 12$

więc:

<span style="font-size: 230%;color:black">

$ {x_{2}}-12{x_{1}} >= 0$



<span style="font-size: 130%;color:#0000ff"> Dodajemy ten warunek do ogólnego zadania i obliczamy Croydon, który ma stosunkowo mało transakcji biznesowych w stosunku do Redhill  

In [45]:
from pulp import *
from fractions import Fraction

# zdefiniowanie problemu- to jest taka platforma do wrzucania na nia wszystkiego

prob = LpProblem("DEA_Croydon2",LpMaximize)

In [46]:
# "Continuous"
# "Integer"

x1=LpVariable("x1",lowBound=0,upBound=None, cat="Integer")
x2=LpVariable("x2",lowBound=0,upBound=None, cat="Integer")
x3=LpVariable("x3",lowBound=0,upBound=None, cat="Integer")

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU

<span style="font-size: 130%;color:#0000ff">$ {125x_{1}} + 50x_{2} \to max$

przy warunku, że dla Croydon:

<span style="font-size: 130%;color:black">$18x_{3}=1$

In [47]:
# objective function
prob += 125*x1 + 50*x2, "Maksymalizacja zysków"

In [48]:
print(prob)

DEA_Croydon2:
MAXIMIZE
125*x1 + 50*x2 + 0
VARIABLES
0 <= x1 Integer
0 <= x2 Integer



##### $
\left\{
\begin{align} 
({125x_{1}} + 50x_{2})-18x_{3} & \leqslant 0& (1)\\
({44x_{1}} + 20x_{2})-16x_{3} & \leqslant 0& (2)\\
({80x_{1}} + 55x_{2})-17x_{3} & \leqslant 0& (3)\\
({23x_{1}} + 12x_{2})-11x_{3} & \leqslant 0& (4)\\
\end{align}
\right. 
$
<span style="font-size: 230%;color:green">

$ {x_{2}}-12{x_{1}} >= 0$


</span>


Warunki znakowe:
#### $
\left\{
\begin{align} 
{x_{1}} & \geqslant 0& (5)\\
{x_{2}} & \geqslant 0& (6)\\
{x_{3}} & \geqslant 0& (7)\\
\end{align}
\right. 
$

$18x_{3}=1$              (8)

In [49]:
# main constraints
prob += (125*x1+50*x2)-18*x3 <= 0
prob += (44*x1+20*x2)-16*x3 <= 0
prob += (80*x1+55*x2)-17*x3 <= 0
prob += (23*x1+12*x2)-11*x3 <= 0
prob += x2-(12*x1)>= 0
prob += 18*x3 == 1

In [50]:
print(prob)

DEA_Croydon2:
MAXIMIZE
125*x1 + 50*x2 + 0
SUBJECT TO
_C1: 125 x1 + 50 x2 - 18 x3 <= 0

_C2: 44 x1 + 20 x2 - 16 x3 <= 0

_C3: 80 x1 + 55 x2 - 17 x3 <= 0

_C4: 23 x1 + 12 x2 - 11 x3 <= 0

_C5: - 12 x1 + x2 >= 0

_C6: 18 x3 = 1

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer



### Rozwiązanie problemu

In [51]:
print("prob.solve",prob.solve())
# status of the solution
print(f"Status: {LpStatus[prob.status]}")

prob.solve -1
Status: Infeasible


In [52]:
for v in prob.variables():
    print(f"{v.name} = {str(Fraction(v.varValue).limit_denominator())}")


x1 = 17/13320
x2 = 17/1110
x3 = 1/18


In [53]:
x1 = 1/125
x2 = 0
x3 = 1/18

In [54]:
(125*x1+50*x2)/(18*x3)

1.0

Ocena Croydon jest bardzo dobra, 100%

Graficznie w artykule mu wyszedł taki ranking:

 1. Croydon 100%
 2. Dorking 43%
 3. Redhill 100%
 4. Reigate 36%

<span style="font-size: 230%;color:#0000ff"> Reigate

In [55]:
from beautifultable import BeautifulTable

table = BeautifulTable()
table.column_headers = ["Bank","Transakcje osobiste" ,"Transakcje biznesowe"
, "Personel"]
table.append_row(["Croydon",    125,   50,     18 ])
table.append_row(["Dorking",    44,    20,     16 ])
table.append_row(["Redhill",    80,    55,     17 ])
table.append_row(["Reigate",    23,    12,     11 ])
print(table)

+---------+---------------------+----------------------+----------+
|  Bank   | Transakcje osobiste | Transakcje biznesowe | Personel |
+---------+---------------------+----------------------+----------+
| Croydon |         125         |          50          |    18    |
+---------+---------------------+----------------------+----------+
| Dorking |         44          |          20          |    16    |
+---------+---------------------+----------------------+----------+
| Redhill |         80          |          55          |    17    |
+---------+---------------------+----------------------+----------+
| Reigate |         23          |          12          |    11    |
+---------+---------------------+----------------------+----------+


In [56]:
from pulp import *
from fractions import Fraction

# zdefiniowanie problemu- to jest taka platforma do wrzucania na nia wszystkiego

prob = LpProblem("DEA_Reigate2",LpMaximize)

In [57]:
# "Continuous"
# "Integer"

x1=LpVariable("x1",lowBound=0,upBound=None, cat="Integer")
x2=LpVariable("x2",lowBound=0,upBound=None, cat="Integer")
x3=LpVariable("x3",lowBound=0,upBound=None, cat="Integer")

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU Reigate

<span style="font-size: 130%;color:#0000ff">$ {23x_{1}} + 12x_{2} \to max$

przy warunku że:

$11x_{3}=1$

In [58]:
# objective function
prob += 23*x1 + 12*x2, "Maksymalizacja zysków"

In [59]:
print(prob)

DEA_Reigate2:
MAXIMIZE
23*x1 + 12*x2 + 0
VARIABLES
0 <= x1 Integer
0 <= x2 Integer



##### $
\left\{
\begin{align} 
({125x_{1}} + 50x_{2})-18x_{3} & \leqslant 0& (1)\\
({44x_{1}} + 20x_{2})-16x_{3} & \leqslant 0& (2)\\
({80x_{1}} + 55x_{2})-17x_{3} & \leqslant 0& (3)\\
({23x_{1}} + 12x_{2})-11x_{3} & \leqslant 0& (4)\\
\end{align}
\right. 
$
<span style="font-size: 230%;color:green">

$ {x_{2}}-12{x_{1}} >= 0$


</span>


Warunki znakowe:
#### $
\left\{
\begin{align} 
{x_{1}} & \geqslant 0& (5)\\
{x_{2}} & \geqslant 0& (6)\\
{x_{3}} & \geqslant 0& (7)\\
\end{align}
\right. 
$

dla Reigate: 

$11x_{3}=1$

In [60]:
# main constraints
prob += (125*x1+50*x2)-18*x3 <= 0
prob += (44*x1+20*x2)-16*x3 <= 0
prob += (80*x1+55*x2)-17*x3 <= 0
prob += (23*x1+12*x2)-11*x3 <= 0
prob += x2-(12*x1)>= 0
prob += 11*x3 == 1

In [61]:
print(prob)

DEA_Reigate2:
MAXIMIZE
23*x1 + 12*x2 + 0
SUBJECT TO
_C1: 125 x1 + 50 x2 - 18 x3 <= 0

_C2: 44 x1 + 20 x2 - 16 x3 <= 0

_C3: 80 x1 + 55 x2 - 17 x3 <= 0

_C4: 23 x1 + 12 x2 - 11 x3 <= 0

_C5: - 12 x1 + x2 >= 0

_C6: 11 x3 = 1

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer



### Rozwiązanie problemu

In [62]:
print("prob.solve",prob.solve())
# status of the solution
print(f"Status: {LpStatus[prob.status]}")

prob.solve -1
Status: Infeasible


In [63]:
for v in prob.variables():
    print(f"{v.name} = {str(Fraction(v.varValue).limit_denominator())}")


x1 = 17/8140
x2 = 51/2035
x3 = 1/11


Kopiuje z góry

In [66]:
x1 = 17/8140
x2 = 51/2035
x3 = 1/11

<span style="font-size: 130%;color:#0000ff"> FUNKCJA CELU

<span style="font-size: 130%;color:#0000ff">$ {23x_{1}} + 12x_{2} \to max$

In [67]:
(23*x1+12*x2)/(11*x3)

0.34877149877149877

0.3617391254394082